In [0]:
# Raw Ingestion from S3
landing_path="s3://databricks-practice-ro/landing/"

sample_data=spark.read.table('samples.nyctaxi.trips').limit(500)

sample_data.write.format("json").mode("overwrite").save(landing_path+"batch_2.json")

In [0]:
from pyspark.sql.functions import *
landing_path='s3://databricks-practice-ro/landing/'
table_path='workspace.default.nycdatset_sample'
df=spark.read.table(table_path)

batch1=df.orderBy(rand(seed=42)).limit(500)
batch1.write.format('json').mode('overwrite').save(landing_path+"batch_1.json")
batch1.count()

In [0]:

 df_bronze=spark.readStream.format('cloudFiles')\
     .option('cloudFiles.format','json')\
         .option('cloudFiles.SchemaLocation','s3://databricks-practice-ro/checkpoints/bronze_schema')\
             .load(landing_path)

df_bronze.writeStream\
    .format('delta')\
        .option('checkpointLocation',"s3://databricks-practice-ro/checkpoints/bronze_data")\
            .outputMode('Append')\
                .trigger(availableNow=True)\
                .table('workspace.default.s3_bronze_taxi')
             

In [0]:
schema_main='s3://databricks-practice-ro/checkpoints/bronze_schema'
checkpoint_main='s3://databricks-practice-ro/checkpoints/bronze_data'
landing_path='s3://databricks-practice-ro/landing/'

s3_bronze=spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','json')\
        .option('cloudFiles.schemaLocation',schema_main)\
            .option('cloudFiles.schemaEvolutionMode','addNewColumns')\
                .option('cloudFiles.rescuedDataColumn','_rescued_data')\
                    .option('cloudFiles.inferColumnTypes','true')\
                        .load(landing_path)

s3_bronze.writeStream.format('delta')\
    .option('checkpointLocation',checkpoint_main)\
        .option('mergeSchema','true')\
        .outputMode('Append')\
            .trigger(availableNow=True)\
                .table('tester.default.bronze_data')

In [0]:
s3_bronzetable=spark.read.table('tester.default.bronze_data')

In [0]:
display(s3_bronzetable)

In [0]:
silver_data=spark.read.table('tester.default.silver_data')
display(silver_data)

In [0]:
%sh

git status